<a href="https://colab.research.google.com/github/katafuchix/at_it_stock_python/blob/main/Ch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install
!pip install Ta-Lib

In [ ]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [ ]:
df = get_stock_data(3021)

In [ ]:
import talib as ta

close = df['Close']

ma5, ma25 = ta.SMA(close, timeperiod=5), ta.SMA(close, timeperiod=25)
df['ma5'], df['ma25'] = ma5, ma25

df.tail()

,Open,High,Low,Close,Volume,ma5,ma25
Date,,,,,,,
2021-09-03,2609.0,2681.0,2590.0,2621.0,48000,2601.6,2665.56
2021-09-06,2693.0,2725.0,2632.0,2683.0,54900,2625.0,2652.48
2021-09-07,2668.0,2701.0,2618.0,2625.0,38100,2631.8,2639.60
2021-09-08,2618.0,2618.0,2491.0,2575.0,61700,2616.8,2629.80
2021-09-09,2554.0,2559.0,2504.0,2548.0,39600,2610.4,2620.72


In [ ]:
import plotly.graph_objs as go

df = df.tail(200) # 200日分のデータ

layout = {
            'title'  : { 'text': "(株)パシフィックネット（3021）", 'x':0.5 }, 
            'xaxis' : { 'title': "日付", 'rangeslider': { 'visible': False } },
            'yaxis' : { 'title': "価格（円）", 'side': "left", 'tickformat': ',' },
            'plot_bgcolor':'light blue'
          }

data =  [
            go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], 
                   increasing_line_color='#00ada9', decreasing_line_color='#a0a0a0')
]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.show()

In [ ]:
import numpy as np

ma5, ma25 = df['ma5'], df['ma25']
cross  = ma5 > ma25

cross_shift = cross.shift(1)
temp_gc = (cross != cross_shift) & (cross == True)
temp_dc  = (cross != cross_shift) & (cross == False)

gc = [m if g == True else np.nan for g, m in zip(temp_gc, ma5)]
dc = [m if d == True else np.nan for d, m in zip(temp_dc, ma25)]
df["gc"], df["dc"] = gc, dc

In [ ]:
import plotly.graph_objs as go

df = df.tail(200)

layout = {
            'title'  : { 'text': "GC", 'x':0.5 }, 
            'xaxis' : { 'title': "日付", 'rangeslider': { 'visible': False } },
            'yaxis' : { 'title': "価格（円）", 'tickformat': ',' },
            'plot_bgcolor':'light blue'
          }

data =  [
    go.Candlestick(name="chart", x = df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], 
                   name="", increasing_line_color= '#00ada9', decreasing_line_color= '#a0a0a0'), 
    go.Scatter(x=df.index, y=df["ma5"], name='MA5', line=dict(color="#ff007f" ,width=1.2)),
    go.Scatter(x=df.index, y=df["ma25"], name='MA25', line=dict(color='#7fbfff' ,width=1.2))
]

fig = go.Figure(data = data, layout = go.Layout(layout))
fig.show()

In [ ]:
import plotly.graph_objs as go

df = df.tail(200)

layout = {
            'title'  : { 'text': "GC", 'x':0.5 }, 
            'xaxis' : { 'title': "日付", 'rangeslider': { 'visible': False } },
            'yaxis' : { 'title': "価格（円）", 'tickformat': ',' },
            'plot_bgcolor':'light blue'
          }

data =  [
    go.Candlestick(name="chart", x = df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], 
                   increasing_line_color= '#00ada9', decreasing_line_color= '#a0a0a0'), 
    go.Scatter(x=df.index, y=df["ma5"], name='MA5', line=dict(color="#ff007f" ,width=1.2)),
    go.Scatter(x=df.index, y=df["ma25"], name='MA25', line=dict(color='#7fbfff' ,width=1.2)), 
    go.Scatter(x=df.index, y=df["gc"], name="Golden Cross", mode='markers', marker=dict(size = 12, color='blueviolet')),
    go.Scatter(x=df.index, y=df["dc"], name="Dead Cross", mode='markers', marker=dict(size = 12, color='black', symbol = 'x'))   
]

fig = go.Figure(data = data, layout = go.Layout(layout))
fig.show()